# React `useRef` Hook 

## Introduction

This lecture explains the `useRef` hook in React, when to use it, how it differs from `useState`, and its two main use cases with practical examples (counter, DOM manipulation, and stopwatch). The session assumes knowledge of other hooks like `useState` and `useEffect` and builds intuition using step‑by‑step code demos. 

***

## Recap: How `useState` Works

React state created with `useState` is designed to drive the UI and trigger re-renders when it changes. 

- A typical counter component:
  - `const [count, setCount] = useState(0);`
  - A button with `onClick={handleIncrement}`.
  - `handleIncrement` calls `setCount(count + 1)`.
  - JSX shows `Count is {count}`. 
- When `setCount` updates the state:
  - React re-renders the component.
  - Wherever `count` is used in the JSX, the value is updated.
  - This behavior can be verified by a `useEffect` without a dependency array that logs on every render. 

**Key points about `useState`:**

- State value is preserved across renders.
- Updating state triggers a component re-render.
- Any logic that depends on state changes will re-run on re-render (e.g., effects). 

***

## Problem: Plain Variables Do Not Persist Across Renders

Using a plain local variable inside a component does not persist its value across re-renders. 

Example idea:

- Inside the component: `let val = 1;`
- In the click handler:
  - Increment `val` and log it.
- Because the component function runs again on each render:
  - `val` is reset to `1` each time.
  - You may see the log always printing `2`, since `val` goes from `1` to `2` on each fresh render. 

**Reason:**

- Every re-render re-executes the component function from the top.
- Local variables are recreated; their previous values are lost.
- So plain variables cannot be used to persist data across renders. 

***

## What `useRef` Is and What It Returns

`useRef` is a React hook that returns a mutable object whose `.current` property can store a value. 

Basic usage pattern:

- `const x = useRef(initialValue);`
- `x` is an object (a “ref object”).
- `x.current` initially holds `initialValue`.
- You can read or write:
  - `x.current = 10;`
  - `console.log(x.current);`. 

**Important properties of refs:**

- The ref object itself is stable between renders (same object identity). 
- Its `.current` value:
  - Persists across renders.
  - Can be mutated without causing any re-render. 

This makes refs suitable for storing data that must survive re-renders but does not need to cause UI updates directly.

***

## Use Case A: Persisting Values Across Renders (Without Re-render)

### Goal

Create a variable whose value persists across renders, but whose changes do not trigger a UI re-render. 

### Steps Explained

1. **Create a ref instead of a plain variable:**

   ```jsx
   const val = useRef(0);
   ```

   - `val` is now an object.
   - `val.current` initially is `0`. 

2. **Update the ref inside a handler:**

   ```jsx
   val.current = val.current + 1;
   console.log("Value of val:", val.current);
   ```

   - Each click increments `val.current`.
   - Value is preserved across renders because the ref object is stable. 

3. **Observe behavior:**

   - Unlike the plain `let val = 1` case, `val.current` keeps increasing: 1, 2, 3, … on successive clicks.
   - The value does not reset on each render. 

4. **No re-render from ref updates:**

   - Changing `val.current` alone does not trigger any re-render.
   - If the component re-renders for some other reason (e.g., some state change), `val.current` still holds the previous value. 

### Summary of Use Case A

- Use `useRef` when:
  - You need to store some data that persists across renders.
  - You do not want changes in that data to cause the UI to re-render. 
- This is different from `useState`, which:
  - Persists across renders.
  - Always triggers a re-render when updated. 

***

## Use Case B (Primary): Direct DOM Access and Manipulation

The main intended use of `useRef` is to access and manipulate DOM elements directly, without `document.getElementById` or similar APIs. 

### Concept

- You create a ref and attach it to a JSX element via the `ref` prop.
- React puts the underlying DOM node into `ref.current`.
- You can then read or modify properties on the DOM node (e.g., styles, focus) using `ref.current`. 

### Example: Changing One Button’s Color from Another Button

**Goal:** When clicking “Change color” button, change the background color of the first button. 

1. **Create two buttons:**

   - Button 1: some normal button (target whose color will change).
   - Button 2: “Change color of first button” with an `onClick` handler. 

2. **Create a ref for the first button:**

   ```jsx
   const buttonRef = useRef(null);
   ```

3. **Attach the ref to the first button:**

   ```jsx
   <button ref={buttonRef}>First Button</button>
   ```

   - Now `buttonRef.current` will point to the actual DOM element of that button. 

4. **Implement the click handler on the second button:**

   ```jsx
   function changeColor() {
     buttonRef.current.style.backgroundColor = "red";
   }
   ```

   - On click, this updates the style of the first button directly via the DOM node reference. 

5. **Wire the handler:**

   ```jsx
   <button onClick={changeColor}>Change color of first button</button>
   ```

### Three-Step Pattern for DOM Manipulation with `useRef`

1. **Create the ref:**

   ```jsx
   const myRef = useRef(null);
   ```

2. **Attach it to a JSX element:**

   ```jsx
   <div ref={myRef}>...</div>
   ```

3. **Use `myRef.current` to manipulate/access:**

   - Change style: `myRef.current.style.backgroundColor = "red";`
   - Focus: `myRef.current.focus();`
   - Call methods: e.g. `videoRef.current.play();` or `videoRef.current.pause();`. 

**Note:**

- This avoids manual DOM querying like `document.getElementById` or `getElementsByClassName` and keeps interactions within React’s model. 

***

## Example: Stopwatch Application Using `useRef`

A more realistic use case combines `useState` and `useRef` to implement a stopwatch (timer) with Start, Stop, and Reset buttons. 

### Requirements

- Display elapsed time in seconds.
- Buttons:
  - Start: begin incrementing time every second.
  - Stop: pause the timer.
  - Reset: stop the timer and set displayed time back to 0. 

### State and Refs

- State:

  ```jsx
  const [time, setTime] = useState(0);
  ```

  - Used to display `time` on UI; changes trigger re-renders to update display. 

- Ref to hold interval ID:

  ```jsx
  const timerRef = useRef(null);
  ```

  - Stores the ID returned by `setInterval`.
  - Needs to persist across re-renders (since `setTime` will cause re-renders).
  - Should not itself trigger re-renders when updated. 

### Start Timer Logic

1. **Start button handler:**

   ```jsx
   function startTimer() {
     // create an interval that runs every 1000ms
     timerRef.current = setInterval(() => {
       setTime(prev => prev + 1);
     }, 1000);
   }
   ```

2. **What happens:**

   - `setInterval` returns an ID (`x`).
   - That ID is stored in `timerRef.current`.
   - Every second, the callback increments `time`.
   - Each `setTime` triggers a re-render, updating the stopwatch display.
   - Across re-renders, `timerRef.current` continues to hold the same interval ID. 

### Stop Timer Logic

1. **Stop button handler:**

   ```jsx
   function stopTimer() {
     clearInterval(timerRef.current);
     timerRef.current = null;
   }
   ```

2. **What happens:**

   - `clearInterval` stops the interval identified by `timerRef.current`.
   - The current displayed `time` value remains as it is (e.g., 27 seconds).
   - Setting `timerRef.current = null` resets the ref’s stored ID but does not trigger a re-render. 

### Reset Timer Logic

1. **Reset button handler:**

   ```jsx
   function resetTimer() {
     clearInterval(timerRef.current); // ensure stopped
     timerRef.current = null;         // reset ref
     setTime(0);                      // reset displayed time
   }
   ```

2. **What happens:**

   - Ensures any running interval is stopped.
   - Resets `time` state to 0, which re-renders the UI to show `0` seconds. 

### Why `useRef` Is Needed Here

- The interval ID returned by `setInterval` must:
  - Survive across renders (because `time` changes cause re-renders).
  - Be accessible in Stop and Reset handlers.
- A plain variable would be re-initialized on every render and would lose the interval ID. 
- `useState` is not ideal:
  - Storing interval ID in state would cause extra re-renders whenever it changes, which are unnecessary.
- `useRef`:
  - Persists the value without triggering re-renders.
  - Perfect for storing such “control handles” (IDs, references to external resources, etc.). 

***

## `useRef` vs `useState` – Quick Comparison

| Aspect                     | `useState`                                           | `useRef`                                                        |
|---------------------------|------------------------------------------------------|------------------------------------------------------------------|
| Persists across renders   | Yes                                                  | Yes                                                              |
| Triggers re-render on set | Yes (`setState` causes re-render)                    | No (changing `.current` does not re-render)                      |
| Typical use               | UI state that must be shown/rendered                 | Mutable values, DOM nodes, timers, focus, etc.                   |
| Access pattern            | `[value, setValue]`                                  | `ref.current`                                                    |
| Good for timers/DOM nodes | Not ideal (unnecessary re-renders)                   | Ideal (store interval IDs, DOM element references, etc.)         | 

***

## Extra Examples Mentioned (From Docs)

The lecture points to React documentation examples demonstrating additional `useRef` patterns: 

- Tracking clicks or render counts using a ref to store a counter without re-rendering on every increment. 
- Focusing inputs:
  - Clicking a button triggers `inputRef.current.focus()`, resulting in the input getting focus and showing a blue focus border. 
- Controlling media:
  - Use `videoRef.current.play()` and `videoRef.current.pause()` to control a video element even if re-renders happen in between. 
- Exposing refs from your own components (advanced pattern) so parent components can manipulate child DOM or behavior. 

These examples reinforce the idea that refs hold mutable values and DOM references that persist independently of render cycles.

***

## Core Takeaways (Summary)

- `useRef` returns a stable object with a `.current` property used to store mutable values. 
- Values stored in `ref.current`:
  - Persist across re-renders.
  - Do not trigger re-renders when they change. 
- **Use Case 1:** Store non-UI data that must survive renders (like counters, interval IDs, flags, etc.) without re-rendering on every change. 
- **Use Case 2 (primary):** Directly access and manipulate DOM elements:
  - Create ref → attach via `ref` prop → use `.current` to change styles, focus, or call methods. 
- Prefer `useState` when changes should update the UI, and `useRef` when you need persistent, mutable values that should not drive re-renders. 